In [4]:
!pip install openml-pytorch

     |████████████████████████████████| 119 kB 5.3 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
  Created wheel for openml-pytorch: filename=openml_pytorch-0.0.5-py3-none-any.whl size=18207 sha256=db916c78c73166901ceb7ecb4965f2f20a672f96920063ffd82f949a3188d77f
  Stored in directory: /root/.cache/pip/wheels/44/4a/9f/7bee098ce5076618f7ebc963f6cee2ca3e76e6ce967797cb31
  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137326 sha256=98802171ff498dc7cc11a01949d824094292d727dbfc2d2bd78def9dab8d6748
  Stored in directory: /root/.cache/pip/wheels/6a/20/88/cf4ac86aa18e2cd647ed16ebe274a5dacee9d0075fa02af250
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=28c347ce583af6c2a690a2d520643210dfe3f2efb05c8b2608eedf1b55233ad5
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built openml-pytorch openml liac-arff


In [12]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import time 
import numpy as np
import random
import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
from torch.utils.data import Dataset
from scipy.stats import zscore
from sklearn.datasets import fetch_openml
import pandas as pd

# torch version 
print(torch.__version__)

1.10.0+cu111


In [13]:
# сама сеть
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, num_layers):
    super().__init__()
    layers = []
    layers.append(nn.Linear(input_size, hidden_size))
    layers.append(nn.ReLU())
    for i in range(num_layers - 2):
      layers.append(nn.Linear(hidden_size, hidden_size))
      layers.append(nn.ReLU())

    layers.append(nn.Linear(hidden_size, output_size))
    self.layers = nn.Sequential(*layers)

  def forward(self, x):
    return self.layers(x)

In [67]:
# получение и предобработка датасета
def get_df_and_lables(num_task):
  task = openml.datasets.get_dataset(num_task)
  df = fetch_openml(name=task.name, as_frame=True)["data"]
  df = df.fillna(df.mean())
  for column in df.columns:
    df[column] = zscore(df[column])
  labels = fetch_openml(name=task.name, as_frame=True)["target"]
  return df, labels

In [95]:
df, labels = get_df_and_lables(40983)

In [96]:
df.head()

,GLCM_Pan,Mean_G,Mean_R,Mean_NIR,SD_Plan
0,-0.482765,-0.409632,0.049840,-0.697603,-0.354813
1,-0.157331,-0.452314,-0.015678,-1.095194,-0.724868
2,0.582671,-0.507868,0.008900,-0.306223,-0.185094
3,0.081103,-0.838533,-0.386084,-1.579725,-0.886132
4,0.637662,-0.544001,-0.058305,0.045682,-0.641235


In [97]:
labels.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: category
Categories (2, object): ['1', '2']

In [98]:
from torch.utils.data import Dataset

class TabularDataset(Dataset):
    def __init__(self, df, target):
        self.df = df
        self.target = target

    def __len__(self):
        return len(df)

    def __getitem__(self, idx):
        return torch.tensor(self.df.loc[idx, :]), int(self.target[idx]) - 1

In [99]:
def normalize(*xs):
    return [None if x is None else F.normalize(x, dim=-1) for x in xs]

In [121]:
import random
def pre_train(f, g, optimizer_f, optimizer_g, num_epochs, dataloaders):
  best_model_wts = f.state_dict()
  best_loss = Inf

  since = time.time()
  for epoch in range(num_epochs):

    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    for phase in ['train', 'val']:
      if phase == 'train':
          f.train(True)
          g.train(True) 
      else:
          f.train(False)
          g.train(False)
      running_loss = 0.0

      for data in dataloaders[phase]:
        inputs, labels = data
        optimizer_f.zero_grad()
        optimizer_g.zero_grad()

        corrupted_batch = inputs.clone().numpy()
        mask = random.sample([i for i in range(inputs.shape[1])], int(inputs.shape[1] * c))
        for i in range(len(inputs)):
          for j in range(len(mask)):
            corrupted_batch[i][mask[j]] = random.choices(np.array(df.iloc[:, mask[j]]))[0]
          
        corrupted_batch = torch.tensor(corrupted_batch)
        if use_gpu:
          inputs = inputs.cuda()
          labels = labels.cuda()
          corrupted_batch = corrupted_batch.cuda()
        z = g(f(inputs.float()))
        z_fake = g(f(corrupted_batch.float()))
        z, z_fake = normalize(z, z_fake)
        logits = torch.matmul(z.T, z_fake)
        labels = torch.arange(len(logits), device="cuda" if use_gpu else "cpu")
        loss = F.cross_entropy(logits, labels, reduction='sum')

        if phase == 'train':
          loss.backward()
          optimizer_f.step()
          optimizer_g.step()
        running_loss += loss.item()
      epoch_loss = running_loss / dataset_sizes[phase]

      print('{} Loss: {:.4f}'.format(phase, epoch_loss))
      if phase == 'val' and epoch_loss < best_loss:
        best_loss = epoch_loss
        best_model_wts = f.state_dict()

    time_elapsed = time.time() - since
    print('Elapsed {:.0f}m {:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60))
  
  return f, best_model_wts

In [122]:
batch_size = 128
use_gpu = torch.cuda.is_available()

dataset= TabularDataset(df = df, target = labels)

train_size = int(np.floor(0.7 * len(dataset)))
val_size = int(np.floor(0.1 * len(dataset)))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

dataloaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": train_size, "val": val_size}

In [126]:
from numpy.core.numeric import Inf
c = 0.6
f = MLP(input_size = df.shape[1], hidden_size= 256, output_size = 256, num_layers=4)
g = MLP(input_size = 256, hidden_size= 256, output_size = df.shape[1], num_layers=2)
optimizer_f = optim.SGD(f.parameters(), lr=0.001)
optimizer_g = optim.SGD(g.parameters(), lr=0.001)


f, best_model_wts = pre_train(f, g, optimizer_f, optimizer_g, 20, dataloaders)

Epoch 0/19
----------
train Loss: 0.0579
val Loss: 0.0411
Elapsed 0m 2s

Epoch 1/19
----------
train Loss: 0.0433
val Loss: 0.0402
Elapsed 0m 4s

Epoch 2/19
----------
train Loss: 0.0412
val Loss: 0.0399
Elapsed 0m 6s

Epoch 3/19
----------
train Loss: 0.0400
val Loss: 0.0395
Elapsed 0m 10s

Epoch 4/19
----------
train Loss: 0.0392
val Loss: 0.0392
Elapsed 0m 13s

Epoch 5/19
----------
train Loss: 0.0385
val Loss: 0.0392
Elapsed 0m 16s

Epoch 6/19
----------
train Loss: 0.0382
val Loss: 0.0390
Elapsed 0m 18s

Epoch 7/19
----------
train Loss: 0.0379
val Loss: 0.0389
Elapsed 0m 21s

Epoch 8/19
----------
train Loss: 0.0379
val Loss: 0.0390
Elapsed 0m 23s

Epoch 9/19
----------
train Loss: 0.0376
val Loss: 0.0388
Elapsed 0m 25s

Epoch 10/19
----------
train Loss: 0.0374
val Loss: 0.0388
Elapsed 0m 27s

Epoch 11/19
----------
train Loss: 0.0374
val Loss: 0.0388
Elapsed 0m 29s

Epoch 12/19
----------
train Loss: 0.0374
val Loss: 0.0388
Elapsed 0m 31s

Epoch 13/19
----------
train Loss: 0.0

In [127]:
def train(f, h, optimizer_f, optimizer_h, num_epochs, dataloaders):
  best_acc = 0
  since = time.time()
  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    for phase in ['train', 'val']:
      if phase == 'train':
          f.train(True)
          h.train(True) 
      else:
          f.train(False)
          h.train(False)
      running_loss = 0.0
      running_corrects = 0

      for data in dataloaders[phase]:
        inputs, labels = data
        optimizer_f.zero_grad()
        optimizer_h.zero_grad()
        if use_gpu:
          inputs = inputs.cuda()
          labels = labels.cuda()
        outputs = h(f(inputs.float()))
        _, preds = torch.max(outputs, 1)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # backward + optimize only if in training phase
        if phase == 'train':
            loss.backward()
            optimizer_f.step()
            optimizer_h.step()
        # statistics
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels).type(torch.float)

    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects / dataset_sizes[phase]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
        phase, epoch_loss, epoch_acc))
    if phase == 'val' and epoch_acc > best_acc:
      best_acc = epoch_acc
  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
      time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

In [128]:
optimizer_f = optim.Adam(f.parameters(), lr=0.001)
optimizer_h = optim.Adam(g.parameters(), lr=0.001)
h = MLP(input_size = 256, hidden_size= 256, output_size = 2, num_layers=2)
train(f, h, optimizer_f, optimizer_h, 10, dataloaders)

Epoch 0/9
----------
val Loss: 0.0013 Acc: 0.9607
Epoch 1/9
----------
val Loss: 0.0010 Acc: 0.9607
Epoch 2/9
----------
val Loss: 0.0007 Acc: 0.9607
Epoch 3/9
----------
val Loss: 0.0004 Acc: 0.9793
Epoch 4/9
----------
val Loss: 0.0003 Acc: 0.9834
Epoch 5/9
----------
val Loss: 0.0004 Acc: 0.9834
Epoch 6/9
----------
val Loss: 0.0003 Acc: 0.9834
Epoch 7/9
----------
val Loss: 0.0004 Acc: 0.9814
Epoch 8/9
----------
val Loss: 0.0003 Acc: 0.9834
Epoch 9/9
----------
val Loss: 0.0003 Acc: 0.9855
Training complete in 0m 12s
Best val Acc: 0.985507


In [62]:
running_corrects = 0
for data in test_loader:
  f.train(False)
  h.train(False)
  inputs, labels = data
  outputs = h(f(inputs.float()))
  _, preds = torch.max(outputs, 1)
  loss = nn.CrossEntropyLoss()(outputs, labels)
  running_corrects += torch.sum(preds == labels).type(torch.float)

acc = running_corrects.numpy() / test_size

print("test accuracy =", acc)

test accuracy = 0.9876160990712074


In [56]:
f = MLP(input_size = df.shape[1], hidden_size= 256, output_size = 256, num_layers=4)
optimizer_f = optim.Adam(f.parameters(), lr=0.001)
optimizer_h = optim.Adam(g.parameters(), lr=0.001)
h = MLP(input_size = 256, hidden_size= 256, output_size = 2, num_layers=2)
train(f, h, optimizer_f, optimizer_h, 10, dataloaders)

Epoch 0/9
----------
val Loss: 0.0015 Acc: 0.9482
Epoch 1/9
----------
val Loss: 0.0013 Acc: 0.9482
Epoch 2/9
----------
val Loss: 0.0009 Acc: 0.9482
Epoch 3/9
----------
val Loss: 0.0006 Acc: 0.9772
Epoch 4/9
----------
val Loss: 0.0005 Acc: 0.9876
Epoch 5/9
----------
val Loss: 0.0005 Acc: 0.9876
Epoch 6/9
----------
val Loss: 0.0003 Acc: 0.9896
Epoch 7/9
----------
val Loss: 0.0003 Acc: 0.9896
Epoch 8/9
----------
val Loss: 0.0004 Acc: 0.9855
Epoch 9/9
----------
val Loss: 0.0002 Acc: 0.9896
Training complete in 0m 11s
Best val Acc: 0.989648


In [57]:
running_corrects = 0
for data in test_loader:
  f.train(False)
  h.train(False)
  inputs, labels = data
  outputs = h(f(inputs.float()))
  _, preds = torch.max(outputs, 1)
  loss = nn.CrossEntropyLoss()(outputs, labels)
  running_corrects += torch.sum(preds == labels).type(torch.float)

acc = running_corrects.numpy() / test_size

print("test accuracy =", acc)

test accuracy = 0.9855521155830753


**Новый датасет**

In [68]:
df, labels = get_df_and_lables(1053)

In [69]:
df.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
0,-0.534225,-0.380102,-0.295508,-0.285373,-0.453283,-0.346848,7.255076,-0.688290,-0.817612,-0.084805,1.663474,-0.084755,-0.406860,-0.081873,-0.263405,0.853939,-0.993538,-0.583329,-0.441735,-0.450430,-0.437880
1,-0.535530,-0.410826,-0.354578,-0.329250,-0.454485,-0.347003,5.386279,-0.704325,-0.826329,-0.084806,1.199350,-0.084767,-0.423636,-0.192883,-0.363730,0.329798,-1.013453,-0.590831,-0.443056,-0.452423,-0.455586
2,0.391486,0.050035,-0.354578,0.219208,0.335124,0.237456,-0.531578,0.327797,0.767373,-0.031788,0.240159,-0.031789,0.415171,0.806207,0.338540,0.329798,0.579778,0.721971,0.289753,0.394830,0.075590
3,1.932161,-0.257206,-0.354578,-0.109867,1.946407,1.895536,-0.469285,0.154084,6.550025,0.086029,1.895537,0.086028,1.723710,2.915398,2.345027,0.853939,0.579778,4.435326,1.722360,2.238852,-0.278527
4,-0.065494,-0.180396,-0.354578,-0.000175,0.046536,-0.038498,-0.469285,0.161032,0.157495,-0.061101,-0.038315,-0.061101,0.029320,-0.192883,0.137892,-0.194344,-0.017684,-0.028201,0.052085,0.035993,-0.189998


In [70]:
labels = labels.replace({"true": 1, "false": 0})

In [71]:
class TabularDataset(Dataset):
    def __init__(self, df, target):
        self.df = df
        self.target = target

    def __len__(self):
        return len(df)

    def __getitem__(self, idx):
        return torch.tensor(self.df.loc[idx, :]), int(self.target[idx])

In [72]:
batch_size = 128
use_gpu = torch.cuda.is_available()
dataset= TabularDataset(df = df, target = labels)

train_size = int(np.floor(0.7 * len(dataset)))
val_size = int(np.floor(0.1 * len(dataset)))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
dataloaders = {"train": train_loader, "val": val_loader}
dataset_sizes = {"train": train_size, "val": val_size}

In [86]:
c = 0.6
t = 1
f = MLP(input_size = df.shape[1], hidden_size= 256, output_size = 256, num_layers=4)
g = MLP(input_size = 256, hidden_size= 256, output_size = df.shape[1], num_layers=2)
optimizer_f = optim.Adam(f.parameters(), lr=0.001)
optimizer_g = optim.Adam(g.parameters(), lr=0.001)

f, best_model_wts = pre_train(f, g, optimizer_f, optimizer_g, 100, dataloaders)

Epoch 0/99
----------
train Loss: 0.3320
val Loss: 0.2744
Elapsed 0m 4s

Epoch 1/99
----------
train Loss: 0.2041
val Loss: 0.2021
Elapsed 0m 7s

Epoch 2/99
----------
train Loss: 0.1710
val Loss: 0.1869
Elapsed 0m 11s

Epoch 3/99
----------
train Loss: 0.1457
val Loss: 0.1558
Elapsed 0m 14s

Epoch 4/99
----------
train Loss: 0.1361
val Loss: 0.1415
Elapsed 0m 18s

Epoch 5/99
----------
train Loss: 0.1225
val Loss: 0.1578
Elapsed 0m 22s

Epoch 6/99
----------
train Loss: 0.1156
val Loss: 0.1416
Elapsed 0m 25s

Epoch 7/99
----------
train Loss: 0.1082
val Loss: 0.1213
Elapsed 0m 29s

Epoch 8/99
----------
train Loss: 0.1033
val Loss: 0.1139
Elapsed 0m 32s

Epoch 9/99
----------
train Loss: 0.0953
val Loss: 0.1143
Elapsed 0m 36s

Epoch 10/99
----------
train Loss: 0.0928
val Loss: 0.1137
Elapsed 0m 40s

Epoch 11/99
----------
train Loss: 0.0916
val Loss: 0.1075
Elapsed 0m 43s

Epoch 12/99
----------
train Loss: 0.0886
val Loss: 0.1079
Elapsed 0m 47s

Epoch 13/99
----------
train Loss: 0.

In [88]:
best_acc = 0
optimizer_f = optim.Adam(f.parameters(), lr=0.001)
optimizer_h = optim.Adam(g.parameters(), lr=0.001)
h = MLP(input_size = 256, hidden_size= 256, output_size = len(labels.unique()), num_layers=2)
train(f, h, optimizer_f, optimizer_h, 20, dataloaders)

Epoch 0/19
----------
val Loss: 0.0037 Acc: 0.8006
Epoch 1/19
----------
val Loss: 0.0037 Acc: 0.8097
Epoch 2/19
----------
val Loss: 0.0038 Acc: 0.8061
Epoch 3/19
----------
val Loss: 0.0037 Acc: 0.8061
Epoch 4/19
----------
val Loss: 0.0037 Acc: 0.8042
Epoch 5/19
----------
val Loss: 0.0036 Acc: 0.8088
Epoch 6/19
----------
val Loss: 0.0036 Acc: 0.8070
Epoch 7/19
----------
val Loss: 0.0037 Acc: 0.8107
Epoch 8/19
----------
val Loss: 0.0036 Acc: 0.8116
Epoch 9/19
----------
val Loss: 0.0036 Acc: 0.8079
Epoch 10/19
----------
val Loss: 0.0036 Acc: 0.8061
Epoch 11/19
----------
val Loss: 0.0037 Acc: 0.8042
Epoch 12/19
----------
val Loss: 0.0038 Acc: 0.8061
Epoch 13/19
----------
val Loss: 0.0038 Acc: 0.8088
Epoch 14/19
----------
val Loss: 0.0040 Acc: 0.8079
Epoch 15/19
----------
val Loss: 0.0037 Acc: 0.8079
Epoch 16/19
----------
val Loss: 0.0038 Acc: 0.8024
Epoch 17/19
----------
val Loss: 0.0039 Acc: 0.8006
Epoch 18/19
----------
val Loss: 0.0038 Acc: 0.8079
Epoch 19/19
----------

In [92]:
running_corrects = 0
for data in test_loader:
  f.train(False)
  h.train(False)
  inputs, labels = data
  outputs = h(f(inputs.float()))
  _, preds = torch.max(outputs, 1)
  loss = nn.CrossEntropyLoss()(outputs, labels)
  running_corrects += torch.sum(preds == labels).type(torch.float)

acc = running_corrects.numpy() / test_size

print("test accuracy =", acc)

test accuracy = 0.8112947658402204


In [93]:
f = MLP(input_size = df.shape[1], hidden_size= 256, output_size = 256, num_layers=4)
optimizer_f = optim.Adam(f.parameters(), lr=0.001)
optimizer_h = optim.Adam(g.parameters(), lr=0.001)
h = MLP(input_size = 256, hidden_size= 256, output_size = 2, num_layers=2)
train(f, h, optimizer_f, optimizer_h, 20, dataloaders)

Epoch 0/19
----------
val Loss: 0.0036 Acc: 0.8051
Epoch 1/19
----------
val Loss: 0.0036 Acc: 0.8051
Epoch 2/19
----------
val Loss: 0.0036 Acc: 0.8051
Epoch 3/19
----------
val Loss: 0.0036 Acc: 0.8051
Epoch 4/19
----------
val Loss: 0.0036 Acc: 0.8061
Epoch 5/19
----------
val Loss: 0.0036 Acc: 0.8079
Epoch 6/19
----------
val Loss: 0.0036 Acc: 0.8079
Epoch 7/19
----------
val Loss: 0.0037 Acc: 0.8051
Epoch 8/19
----------
val Loss: 0.0036 Acc: 0.8042
Epoch 9/19
----------
val Loss: 0.0037 Acc: 0.8070
Epoch 10/19
----------
val Loss: 0.0037 Acc: 0.8051
Epoch 11/19
----------
val Loss: 0.0038 Acc: 0.8079
Epoch 12/19
----------
val Loss: 0.0037 Acc: 0.8097
Epoch 13/19
----------
val Loss: 0.0037 Acc: 0.8088
Epoch 14/19
----------
val Loss: 0.0038 Acc: 0.8125
Epoch 15/19
----------
val Loss: 0.0037 Acc: 0.8051
Epoch 16/19
----------
val Loss: 0.0038 Acc: 0.8208
Epoch 17/19
----------
val Loss: 0.0038 Acc: 0.8107
Epoch 18/19
----------
val Loss: 0.0037 Acc: 0.8153
Epoch 19/19
----------

In [94]:
running_corrects = 0
for data in test_loader:
  f.train(False)
  h.train(False)
  inputs, labels = data
  outputs = h(f(inputs.float()))
  _, preds = torch.max(outputs, 1)
  loss = nn.CrossEntropyLoss()(outputs, labels)
  running_corrects += torch.sum(preds == labels).type(torch.float)

acc = running_corrects.numpy() / test_size

print("test accuracy =", acc)

test accuracy = 0.8089990817263545


In [81]:
!pip install pytorch-tabnet

**TABnet**

In [83]:
from sklearn.model_selection import train_test_split

df, labels = get_df_and_lables(1053)
x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.125)

In [84]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

clf = TabNetClassifier()
clf.fit(
  x_train.to_numpy(), y_train.to_numpy(),
  eval_set=[(x_val.to_numpy(), y_val.to_numpy())]
)
preds = clf.predict(x_test.to_numpy())

Device used : cpu
epoch 0  | loss: 0.73328 | val_0_auc: 0.52752 |  0:00:00s
epoch 1  | loss: 0.55522 | val_0_auc: 0.63843 |  0:00:00s
epoch 2  | loss: 0.51281 | val_0_auc: 0.67008 |  0:00:01s
epoch 3  | loss: 0.48332 | val_0_auc: 0.67762 |  0:00:01s
epoch 4  | loss: 0.46382 | val_0_auc: 0.67196 |  0:00:02s
epoch 5  | loss: 0.46514 | val_0_auc: 0.66598 |  0:00:02s
epoch 6  | loss: 0.46671 | val_0_auc: 0.6535  |  0:00:02s
epoch 7  | loss: 0.46548 | val_0_auc: 0.68028 |  0:00:03s
epoch 8  | loss: 0.45969 | val_0_auc: 0.67721 |  0:00:03s
epoch 9  | loss: 0.45876 | val_0_auc: 0.68215 |  0:00:04s
epoch 10 | loss: 0.45917 | val_0_auc: 0.67423 |  0:00:04s
epoch 11 | loss: 0.45835 | val_0_auc: 0.68116 |  0:00:04s
epoch 12 | loss: 0.45554 | val_0_auc: 0.68113 |  0:00:05s
epoch 13 | loss: 0.45415 | val_0_auc: 0.6841  |  0:00:05s
epoch 14 | loss: 0.45816 | val_0_auc: 0.6838  |  0:00:06s
epoch 15 | loss: 0.4536  | val_0_auc: 0.69162 |  0:00:06s
epoch 16 | loss: 0.45216 | val_0_auc: 0.69377 |  0:00:

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

0.8089113458888378